# imports

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
tf.enable_eager_execution()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
#!pip install tensorflow_datasets

# load and preprocess data

In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised = True)

Instructions for updating:
Colocations handled automatically by placer.


W0919 23:03:31.143882  3448 deprecation.py:323] From C:\Users\Lindsay\Anaconda3\envs\py36\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:423: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
W0919 23:03:31.388738  3448 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [4]:
#!pip install tensorflow==2.0.0-beta1

In [5]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)



In [6]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label


In [7]:
scaled_train_and_validation_data = mnist_train.map(scale)

In [8]:
test_data = mnist_test.map(scale)

In [9]:
buffer_size = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


In [10]:
batch_size = 100
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# outline the model

In [11]:
input_size = 784
output_size = 10
hidden_layer_size =100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape =(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax')            
                            ])

# optimizer and loss function

In [12]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# training

In [14]:
num_epochs = 5
model.fit(train_data, epochs = num_epochs, validation_data = (validation_inputs, validation_targets), verbose = 4, steps_per_epoch = 100)

Epoch 1/5
6000/6000 [==============================] - ETA: 15s - loss: 0.3615 - acc: 0.86 - ETA: 1s - loss: 0.3538 - acc: 0.8890 - ETA: 1s - loss: 0.3528 - acc: 0.890 - ETA: 1s - loss: 0.3587 - acc: 0.889 - ETA: 0s - loss: 0.3619 - acc: 0.888 - ETA: 0s - loss: 0.3555 - acc: 0.888 - ETA: 0s - loss: 0.3634 - acc: 0.889 - ETA: 0s - loss: 0.3568 - acc: 0.891 - ETA: 0s - loss: 0.3475 - acc: 0.894 - ETA: 0s - loss: 0.3544 - acc: 0.895 - ETA: 0s - loss: 0.3540 - acc: 0.895 - ETA: 0s - loss: 0.3526 - acc: 0.895 - 1s 149us/sample - loss: 0.3553 - acc: 0.8950
Epoch 2/5
6000/6000 [==============================] - ETA: 0s - loss: 0.1969 - acc: 0.960 - ETA: 0s - loss: 0.2096 - acc: 0.945 - ETA: 0s - loss: 0.2472 - acc: 0.933 - ETA: 0s - loss: 0.2479 - acc: 0.936 - ETA: 0s - loss: 0.2762 - acc: 0.922 - ETA: 0s - loss: 0.2668 - acc: 0.923 - ETA: 0s - loss: 0.2728 - acc: 0.922 - ETA: 0s - loss: 0.2677 - acc: 0.924 - ETA: 0s - loss: 0.2608 - acc: 0.925 - ETA: 0s - loss: 0.2647 - acc: 0.924 - ETA: 0s 

# test the model

In [17]:
test_loss, test_accuracy = model.evaluate(test_data, steps = 100)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

  1/100 [..............................] - ETA: 4:05 - loss: 0.1730 - acc: 0.9496WARNING:tensorflow:Your dataset iterator ran out of data.


W0919 23:09:17.463999  3448 training_generator.py:261] Your dataset iterator ran out of data.


Test loss: 0.00. Test accuracy: 94.96%
